In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
L, T = 1, 1
c = 2.

Nx = 100
Nt = 800

tau = T / Nt
h = L / Nx
gamma = (c * tau / h)**2

print('gamma = ', gamma)

# set the grids
y = np.zeros((Nt+1, Nx+1), dtype=float)  # the solution

xj = np.linspace(0, 1, Nx+1, endpoint=True)
tn = np.linspace(0, 1, Nt+1, endpoint=True)

gamma =  0.0625


In [3]:
# initial conditions
def u0(x):
    return np.sin(2.*np.pi*x)

def u0p(x):
    return 0.01

In [4]:
def make_step(y, n, gamma):
    """Make a step n -> n+1."""
    assert n >= 1
    y[n+1, 0] = 0
    y[n+1, -1] = 0
    for j in range(1, Nx):
        y[n+1, j] = 2*y[n, j] - y[n-1, j] + gamma * (y[n, j+1] - 2*y[n, j] + y[n, j-1])

# set initial conditions
y[0, :] = u0(xj)
y[1, :] = y[0, :] + tau * u0p(xj)
for n in range(1, Nt):
    make_step(y, n, gamma)

In [5]:
%%capture
# draw / animate
# http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-as-interactive-javascript-widgets/

def init():
    line.set_data([], [])
    return line,

def animate(n):
    line.set_data(xj, y[n, :])
    return line,

from matplotlib import animation, rc
from IPython.display import HTML
%matplotlib inline

fig, ax = plt.subplots()
ax.set_xlim([0, 1])
ax.set_ylim([-1.1, 1.1])
line, = ax.plot([], [], '.-', lw=2)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=Nt, interval=20, 
                               blit=True);

In [ ]:
HTML(anim.to_jshtml())

# Задание 1

In [ ]:
answer = np.empty((Nt+1, Nx+1))
u = lambda x, t: 0.5*(np.sin(2*np.pi*(x-2*t)) + np.cos(2*np.pi*(x+2*t-0.25)))

for n in range(Nt):
    answer[n] = u(xj, tn[n])

In [ ]:
%%capture
# draw / animate
# http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-as-interactive-javascript-widgets/

def init():
    line.set_data([], [])
    line2.set_data([], [])
    return line, line2,

def animate(n):
    line.set_data(xj, y[n, :])
    line2.set_data(xj, answer[n, :])
    return line, line2,

fig, ax = plt.subplots()
ax.set_xlim([0, 1])
ax.set_ylim([-1.1, 1.1])
line, = ax.plot([], [], '.-', lw=2)
line2, = ax.plot([], [], '.-', lw=2)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=Nt, interval=20, 
                               blit=True);

In [ ]:
HTML(anim.to_jshtml())

## h

In [ ]:
Nxs = np.array(list(range(50, 400, 15)))
Nt = 800
tn = np.linspace(0, 1, Nt+1, endpoint=True)
tau = T / Nt
error = []

for Nx in Nxs:
  xj = np.linspace(0, 1, Nx+1, endpoint=True)
  h = L / Nx
  gamma = (c * tau / h)**2

  y = np.empty((Nt+1, Nx+1))
  answer = np.empty((Nt+1, Nx+1))

  y[0] = u0(xj)
  y[1] = y[0] + tau * u0p(xj)
  answer[0] = u(xj, tn[0])
  answer[Nt] = u(xj, tn[Nt])

  for n in range(1, Nt):
      make_step(y, n, gamma)
      answer[n] = u(xj, tn[n])
  
  diff = np.abs(answer[-1] - y[-1])
  error.append(np.sum(diff[1:-1]) + (diff[0] + diff[-1]) / 2)
  error[-1] *= h

In [ ]:
plt.plot(L / Nxs, error, marker='o')
plt.xscale('log')
plt.yscale('log')
plt.show()

## tau

In [ ]:
Nts = np.array(list(range(350, 1300, 25)))
Nx = 100
xj = np.linspace(0, 1, Nx+1, endpoint=True)
h = L / Nx
error = []

for Nt in Nts:
  tn = np.linspace(0, 1, Nt+1, endpoint=True)
  tau = T / Nt
  gamma = (c * tau / h)**2

  y = np.empty((Nt+1, Nx+1))
  answer = np.empty((Nt+1, Nx+1))

  y[0] = u0(xj)
  y[1] = y[0] + tau * u0p(xj)
  answer[0] = u(xj, tn[0])
  answer[Nt] = u(xj, tn[Nt])

  for n in range(1, Nt):
      make_step(y, n, gamma)
      answer[n] = u(xj, tn[n])
  
  diff = np.abs(answer[-1] - y[-1])
  error.append(np.sum(diff[1:-1]) + (diff[0] + diff[-1]) / 2)
  error[-1] *= h

In [ ]:
plt.plot(T / Nts, error, marker='o')
plt.xscale('log')
plt.yscale('log')
plt.show()

# Задание 2

In [ ]:
def u(x, t):
  return np.exp(-25*(x-1.2-t)**2) * np.sin((x-1.2-t)/0.03)

def u0(x):
  return u(x, 0)
  
def u0p(x):
  return (u(x, 0.0001) - u(x, 0))/0.0001
    
def make_step(y, n, gamma):
    assert n >= 1
    y[n+1, 0] = 2*y[n, 0] - y[n-1, 0] + gamma * (y[n, 1] - 2*y[n, 0] + y[n, -1])
    y[n+1, -1] = 2*y[n, -1] - y[n-1, -1] + gamma * (y[n, 0] - 2*y[n, -1] + y[n, -2])
    for j in range(1, Nx):
        y[n+1, j] = 2*y[n, j] - y[n-1, j] + gamma * (y[n, j+1] - 2*y[n, j] + y[n, j-1])

L, T = 3, 3
c = 1

Nx = 300
Nt = 900

tau = T / Nt
h = L / Nx
gamma = (c * tau / h)**2

y = np.empty((Nt+1, Nx+1), dtype=float)
xj = np.linspace(0, L, Nx+1, endpoint=True)
tn = np.linspace(0, T, Nt+1, endpoint=True)

y[0] = u0(xj)
y[1] = y[0] + tau * u0p(xj)
for n in range(1, Nt):
    make_step(y, n, gamma)

In [ ]:
%%capture
def init():
    line.set_data([], [])
    return line,

def animate(n):
    line.set_data(xj, y[n, :])
    return line,

fig, ax = plt.subplots()
ax.set_xlim([0, L])
ax.set_ylim([-1.1, 1.1])
line, = ax.plot([], [], '.-', lw=2)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=Nt, interval=20, 
                               blit=True);

In [ ]:
HTML(anim.to_jshtml())